### Code submission for Random Forest (without PCA) and Multilayer Perceptron

### Data loading

In [3]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
#!pip install functools
import functools
import glob
from functools import reduce
from pyspark.sql import SQLContext
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext
sqlc = SQLContext(sc)

In [8]:
# Import the data 
test_sparkdf = spark.read.csv("./emnist-byclass-test.csv", inferSchema=True, header=True).toDF(*col_list)
train_sparkdf = spark.read.csv("./emnist-byclass-train.csv", inferSchema=True, header=True).toDF(*col_list)

### Pre Processing

In [7]:
# Build new column name list
col_list = ["label"]
i=1
while i<785:
    col_list.append("pixel_"+str(i))
    i=i+1
print(col_list)

['label', 'pixel_1', 'pixel_2', 'pixel_3', 'pixel_4', 'pixel_5', 'pixel_6', 'pixel_7', 'pixel_8', 'pixel_9', 'pixel_10', 'pixel_11', 'pixel_12', 'pixel_13', 'pixel_14', 'pixel_15', 'pixel_16', 'pixel_17', 'pixel_18', 'pixel_19', 'pixel_20', 'pixel_21', 'pixel_22', 'pixel_23', 'pixel_24', 'pixel_25', 'pixel_26', 'pixel_27', 'pixel_28', 'pixel_29', 'pixel_30', 'pixel_31', 'pixel_32', 'pixel_33', 'pixel_34', 'pixel_35', 'pixel_36', 'pixel_37', 'pixel_38', 'pixel_39', 'pixel_40', 'pixel_41', 'pixel_42', 'pixel_43', 'pixel_44', 'pixel_45', 'pixel_46', 'pixel_47', 'pixel_48', 'pixel_49', 'pixel_50', 'pixel_51', 'pixel_52', 'pixel_53', 'pixel_54', 'pixel_55', 'pixel_56', 'pixel_57', 'pixel_58', 'pixel_59', 'pixel_60', 'pixel_61', 'pixel_62', 'pixel_63', 'pixel_64', 'pixel_65', 'pixel_66', 'pixel_67', 'pixel_68', 'pixel_69', 'pixel_70', 'pixel_71', 'pixel_72', 'pixel_73', 'pixel_74', 'pixel_75', 'pixel_76', 'pixel_77', 'pixel_78', 'pixel_79', 'pixel_80', 'pixel_81', 'pixel_82', 'pixel_83', 'pi

In [4]:
# Print the imported data's first rows
print(test_sparkdf.take(1))
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(train_sparkdf.take(1))

[Row(label=36, pixel_1=0, pixel_2=0, pixel_3=0, pixel_4=0, pixel_5=0, pixel_6=0, pixel_7=0, pixel_8=0, pixel_9=0, pixel_10=0, pixel_11=0, pixel_12=0, pixel_13=0, pixel_14=0, pixel_15=0, pixel_16=0, pixel_17=0, pixel_18=0, pixel_19=0, pixel_20=0, pixel_21=0, pixel_22=0, pixel_23=0, pixel_24=0, pixel_25=0, pixel_26=0, pixel_27=0, pixel_28=0, pixel_29=0, pixel_30=0, pixel_31=0, pixel_32=0, pixel_33=0, pixel_34=0, pixel_35=0, pixel_36=0, pixel_37=0, pixel_38=0, pixel_39=0, pixel_40=0, pixel_41=0, pixel_42=0, pixel_43=0, pixel_44=0, pixel_45=0, pixel_46=0, pixel_47=0, pixel_48=0, pixel_49=0, pixel_50=0, pixel_51=0, pixel_52=0, pixel_53=0, pixel_54=0, pixel_55=0, pixel_56=0, pixel_57=0, pixel_58=0, pixel_59=0, pixel_60=0, pixel_61=0, pixel_62=0, pixel_63=0, pixel_64=0, pixel_65=0, pixel_66=0, pixel_67=0, pixel_68=0, pixel_69=0, pixel_70=0, pixel_71=0, pixel_72=0, pixel_73=1, pixel_74=20, pixel_75=37, pixel_76=37, pixel_77=37, pixel_78=37, pixel_79=32, pixel_80=7, pixel_81=0, pixel_82=0, pixe

[Row(label=36, pixel_1=0, pixel_2=0, pixel_3=0, pixel_4=0, pixel_5=0, pixel_6=0, pixel_7=0, pixel_8=0, pixel_9=0, pixel_10=0, pixel_11=0, pixel_12=0, pixel_13=0, pixel_14=0, pixel_15=0, pixel_16=0, pixel_17=0, pixel_18=0, pixel_19=0, pixel_20=0, pixel_21=0, pixel_22=0, pixel_23=0, pixel_24=0, pixel_25=0, pixel_26=0, pixel_27=0, pixel_28=0, pixel_29=0, pixel_30=0, pixel_31=0, pixel_32=0, pixel_33=0, pixel_34=0, pixel_35=0, pixel_36=0, pixel_37=0, pixel_38=0, pixel_39=0, pixel_40=0, pixel_41=0, pixel_42=0, pixel_43=0, pixel_44=0, pixel_45=0, pixel_46=0, pixel_47=0, pixel_48=0, pixel_49=0, pixel_50=0, pixel_51=0, pixel_52=0, pixel_53=0, pixel_54=0, pixel_55=0, pixel_56=0, pixel_57=0, pixel_58=0, pixel_59=0, pixel_60=0, pixel_61=0, pixel_62=0, pixel_63=0, pixel_64=0, pixel_65=0, pixel_66=0, pixel_67=0, pixel_68=0, pixel_69=0, pixel_70=0, pixel_71=0, pixel_72=0, pixel_73=0, pixel_74=0, pixel_75=1, pixel_76=19, pixel_77=27, pixel_78=8, pixel_79=0, pixel_80=0, pixel_81=0, pixel_82=0, pixel_83

In [10]:
# Scaled the pixel values by dividing them by the maximum value (255), and changed the name of the colums
train_scaled = train_sparkdf
spark_list = col_list[1:]
for c in spark_list:
    name = str(c)
    train_scaled = train_scaled.withColumn(name, col(name)/255)
    
test_scaled = test_sparkdf
for c in spark_list:
    name = str(c)
    test_scaled = test_scaled.withColumn(name, col(name)/255)
    
print(train_scaled.take(1))
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(test_scaled.take(1))

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[Row(label=36, pixel_1=0.0, pixel_2=0.0, pixel_3=0.0, pixel_4=0.0, pixel_5=0.0, pixel_6=0.0, pixel_7=0.0, pixel_8=0.0, pixel_9=0.0, pixel_10=0.0, pixel_11=0.0, pixel_12=0.0, pixel_13=0.0, pixel_14=0.0, pixel_15=0.0, pixel_16=0.0, pixel_17=0.0, pixel_18=0.0, pixel_19=0.0, pixel_20=0.0, pixel_21=0.0, pixel_22=0.0, pixel_23=0.0, pixel_24=0.0, pixel_25=0.0, pixel_26=0.0, pixel_27=0.0, pixel_28=0.0, pixel_29=0.0, pixel_30=0.0, pixel_31=0.0, pixel_32=0.0, pixel_33=0.0, pixel_34=0.0, pixel_35=0.0, pixel_36=0.0, pixel_37=0.0, pixel_38=0.0, pixel_39=0.0, pixel_40=0.0, pixel_41=0.0, pixel_42=0.0, pixel_43=0.0, pixel_44=0.0, pixel_45=0.0, pixel_46=0.0, pixel_47=0.0, pixel_48=0.0, pixel_49=0.0, pixel_50=0.0, pixel_51=0.0, pixel_52=0.0, pixel_53=0.0, pixel_54=0.0, pixel_55=0.0, pixel_56=0.0, pixel_57=0.0, pixel_58=0.0, pixel_59=0.0, pixel_60=0.0, pixel_61=0.0, pixel_62=0.0, pixel_63=0.0, pixel_64=0.0, pixel_65=0.0, pixel_66=0.0, pixel_67=0.0, pixel

In [6]:
# Split the training data into train (80%) and validation (20%), save them
train, validation = train_scaled.randomSplit([0.8,0.2], seed=0)

train.write.format('com.databricks.spark.csv').save('train.csv')
validation.write.format('com.databricks.spark.csv').save('validation.csv')

In [7]:
# Print the new dataframes
print(train.take(1))
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(validation.take(1))

[Row(label=0, pixel_1=0.0, pixel_2=0.0, pixel_3=0.0, pixel_4=0.0, pixel_5=0.0, pixel_6=0.0, pixel_7=0.0, pixel_8=0.0, pixel_9=0.0, pixel_10=0.0, pixel_11=0.0, pixel_12=0.0, pixel_13=0.0, pixel_14=0.0, pixel_15=0.0, pixel_16=0.0, pixel_17=0.0, pixel_18=0.0, pixel_19=0.0, pixel_20=0.0, pixel_21=0.0, pixel_22=0.0, pixel_23=0.0, pixel_24=0.0, pixel_25=0.0, pixel_26=0.0, pixel_27=0.0, pixel_28=0.0, pixel_29=0.0, pixel_30=0.0, pixel_31=0.0, pixel_32=0.0, pixel_33=0.0, pixel_34=0.0, pixel_35=0.0, pixel_36=0.0, pixel_37=0.0, pixel_38=0.0, pixel_39=0.0, pixel_40=0.0, pixel_41=0.0, pixel_42=0.0, pixel_43=0.0, pixel_44=0.0, pixel_45=0.0, pixel_46=0.0, pixel_47=0.0, pixel_48=0.0, pixel_49=0.0, pixel_50=0.0, pixel_51=0.0, pixel_52=0.0, pixel_53=0.0, pixel_54=0.0, pixel_55=0.0, pixel_56=0.0, pixel_57=0.0, pixel_58=0.0, pixel_59=0.0, pixel_60=0.0, pixel_61=0.0, pixel_62=0.0, pixel_63=0.0, pixel_64=0.0, pixel_65=0.0, pixel_66=0.0, pixel_67=0.0, pixel_68=0.0, pixel_69=0.0, pixel_70=0.0, pixel_71=0.0, p

[Row(label=0, pixel_1=0.0, pixel_2=0.0, pixel_3=0.0, pixel_4=0.0, pixel_5=0.0, pixel_6=0.0, pixel_7=0.0, pixel_8=0.0, pixel_9=0.0, pixel_10=0.0, pixel_11=0.0, pixel_12=0.0, pixel_13=0.0, pixel_14=0.0, pixel_15=0.0, pixel_16=0.0, pixel_17=0.0, pixel_18=0.0, pixel_19=0.0, pixel_20=0.0, pixel_21=0.0, pixel_22=0.0, pixel_23=0.0, pixel_24=0.0, pixel_25=0.0, pixel_26=0.0, pixel_27=0.0, pixel_28=0.0, pixel_29=0.0, pixel_30=0.0, pixel_31=0.0, pixel_32=0.0, pixel_33=0.0, pixel_34=0.0, pixel_35=0.0, pixel_36=0.0, pixel_37=0.0, pixel_38=0.0, pixel_39=0.0, pixel_40=0.0, pixel_41=0.0, pixel_42=0.0, pixel_43=0.0, pixel_44=0.0, pixel_45=0.0, pixel_46=0.0, pixel_47=0.0, pixel_48=0.0, pixel_49=0.0, pixel_50=0.0, pixel_51=0.0, pixel_52=0.0, pixel_53=0.0, pixel_54=0.0, pixel_55=0.0, pixel_56=0.0, pixel_57=0.0, pixel_58=0.0, pixel_59=0.0, pixel_60=0.0, pixel_61=0.0, pixel_62=0.0, pixel_63=0.0, pixel_64=0.0, pixel_65=0.0, pixel_66=0.0, pixel_67=0.0, pixel_68=0.0, pixel_69=0.0, pixel_70=0.0, pixel_71=0.0, p

In [2]:
# Import the train data from the saved csv file created above
files = glob.glob("./train.csv/*.csv")
train_com = reduce(lambda x,y: x.unionAll(y),[sqlc.read.format('com.databricks.spark.csv').load(f,header='false',inferSchema='true') for f in files])

In [3]:
# Import the validation data from the saved csv file created above
files1 = glob.glob("./validation.csv/*.csv")
validation_com = reduce(lambda x,y: x.unionAll(y),[sqlc.read.format('com.databricks.spark.csv').load(f,header='false',inferSchema='true') for f in files1])

In [11]:
# Printed the counts and type of dataframes
print(train_com.count(), len(train_com.columns))
print(type(train_com))
print(validation_com.count(),len(validation_com.columns))
print(type(validation_com))
print(test_scaled.count(),len(test_scaled.columns))
print(type(test_scaled))

558402 785
<class 'pyspark.sql.dataframe.DataFrame'>
139529 785
<class 'pyspark.sql.dataframe.DataFrame'>
116322 785
<class 'pyspark.sql.dataframe.DataFrame'>


In [12]:
# Changed the names of the columns as per the column list
old_cols_train = train_com.schema.names
old_cols_validation = validation_com.schema.names

validation_newdf = reduce(lambda validation_com, i: validation_com.withColumnRenamed(old_cols_validation[i],col_list[i]), range(len(old_cols_validation)), validation_com)

train_newdf = reduce(lambda train_com, i: train_com.withColumnRenamed(old_cols_train[i],col_list[i]), range(len(old_cols_train)), train_com)
print(train_newdf.take(1))
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(validation_newdf.take(1))
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(test_scaled.take(1))

[Row(label=0, pixel_1=0.0, pixel_2=0.0, pixel_3=0.0, pixel_4=0.0, pixel_5=0.0, pixel_6=0.0, pixel_7=0.0, pixel_8=0.0, pixel_9=0.0, pixel_10=0.0, pixel_11=0.0, pixel_12=0.0, pixel_13=0.0, pixel_14=0.0, pixel_15=0.0, pixel_16=0.0, pixel_17=0.0, pixel_18=0.0, pixel_19=0.0, pixel_20=0.0, pixel_21=0.0, pixel_22=0.0, pixel_23=0.0, pixel_24=0.0, pixel_25=0.0, pixel_26=0.0, pixel_27=0.0, pixel_28=0.0, pixel_29=0.0, pixel_30=0.0, pixel_31=0.0, pixel_32=0.0, pixel_33=0.0, pixel_34=0.0, pixel_35=0.0, pixel_36=0.0, pixel_37=0.0, pixel_38=0.0, pixel_39=0.0, pixel_40=0.0, pixel_41=0.0, pixel_42=0.0, pixel_43=0.0, pixel_44=0.0, pixel_45=0.0, pixel_46=0.0, pixel_47=0.0, pixel_48=0.0, pixel_49=0.0, pixel_50=0.0, pixel_51=0.0, pixel_52=0.0, pixel_53=0.0, pixel_54=0.0, pixel_55=0.0, pixel_56=0.0, pixel_57=0.0, pixel_58=0.0, pixel_59=0.0, pixel_60=0.0, pixel_61=0.0, pixel_62=0.0, pixel_63=0.0, pixel_64=0.0, pixel_65=0.0, pixel_66=0.0, pixel_67=0.0, pixel_68=0.0, pixel_69=0.0, pixel_70=0.0, pixel_71=0.0, p

[Row(label=0, pixel_1=0.0, pixel_2=0.0, pixel_3=0.0, pixel_4=0.0, pixel_5=0.0, pixel_6=0.0, pixel_7=0.0, pixel_8=0.0, pixel_9=0.0, pixel_10=0.0, pixel_11=0.0, pixel_12=0.0, pixel_13=0.0, pixel_14=0.0, pixel_15=0.0, pixel_16=0.0, pixel_17=0.0, pixel_18=0.0, pixel_19=0.0, pixel_20=0.0, pixel_21=0.0, pixel_22=0.0, pixel_23=0.0, pixel_24=0.0, pixel_25=0.0, pixel_26=0.0, pixel_27=0.0, pixel_28=0.0, pixel_29=0.0, pixel_30=0.0, pixel_31=0.0, pixel_32=0.0, pixel_33=0.0, pixel_34=0.0, pixel_35=0.0, pixel_36=0.0, pixel_37=0.0, pixel_38=0.0, pixel_39=0.0, pixel_40=0.0, pixel_41=0.0, pixel_42=0.0, pixel_43=0.0, pixel_44=0.0, pixel_45=0.0, pixel_46=0.0, pixel_47=0.0, pixel_48=0.0, pixel_49=0.0, pixel_50=0.0, pixel_51=0.0, pixel_52=0.0, pixel_53=0.0, pixel_54=0.0, pixel_55=0.0, pixel_56=0.0, pixel_57=0.0, pixel_58=0.0, pixel_59=0.0, pixel_60=0.0, pixel_61=0.0, pixel_62=0.0, pixel_63=0.0, pixel_64=0.0, pixel_65=0.0, pixel_66=0.0, pixel_67=0.0, pixel_68=0.0, pixel_69=0.0, pixel_70=0.0, pixel_71=0.0, p

[Row(label=36, pixel_1=0.0, pixel_2=0.0, pixel_3=0.0, pixel_4=0.0, pixel_5=0.0, pixel_6=0.0, pixel_7=0.0, pixel_8=0.0, pixel_9=0.0, pixel_10=0.0, pixel_11=0.0, pixel_12=0.0, pixel_13=0.0, pixel_14=0.0, pixel_15=0.0, pixel_16=0.0, pixel_17=0.0, pixel_18=0.0, pixel_19=0.0, pixel_20=0.0, pixel_21=0.0, pixel_22=0.0, pixel_23=0.0, pixel_24=0.0, pixel_25=0.0, pixel_26=0.0, pixel_27=0.0, pixel_28=0.0, pixel_29=0.0, pixel_30=0.0, pixel_31=0.0, pixel_32=0.0, pixel_33=0.0, pixel_34=0.0, pixel_35=0.0, pixel_36=0.0, pixel_37=0.0, pixel_38=0.0, pixel_39=0.0, pixel_40=0.0, pixel_41=0.0, pixel_42=0.0, pixel_43=0.0, pixel_44=0.0, pixel_45=0.0, pixel_46=0.0, pixel_47=0.0, pixel_48=0.0, pixel_49=0.0, pixel_50=0.0, pixel_51=0.0, pixel_52=0.0, pixel_53=0.0, pixel_54=0.0, pixel_55=0.0, pixel_56=0.0, pixel_57=0.0, pixel_58=0.0, pixel_59=0.0, pixel_60=0.0, pixel_61=0.0, pixel_62=0.0, pixel_63=0.0, pixel_64=0.0, pixel_65=0.0, pixel_66=0.0, pixel_67=0.0, pixel_68=0.0, pixel_69=0.0, pixel_70=0.0, pixel_71=0.0, 

In [15]:
# Print the counts, types, and the first rows of the dataframes
print(train_newdf.count(), len(train_newdf.columns))
print(type(train_newdf))
print(validation_newdf.count(),len(validation_newdf.columns))
print(type(validation_newdf))
print(test_scaled.count(),len(test_scaled.columns))
print(type(test_scaled))
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(train_newdf.take(1))
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(train_newdf.groupBy('label').count().show())
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(validation_newdf.take(1))
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(validation_newdf.groupBy('label').count().show())
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(test_scaled.take(1))
print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
print(test_scaled.groupBy('label').count().show())

558402 785
<class 'pyspark.sql.dataframe.DataFrame'>
139529 785
<class 'pyspark.sql.dataframe.DataFrame'>
116322 785
<class 'pyspark.sql.dataframe.DataFrame'>
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[Row(label=0, pixel_1=0.0, pixel_2=0.0, pixel_3=0.0, pixel_4=0.0, pixel_5=0.0, pixel_6=0.0, pixel_7=0.0, pixel_8=0.0, pixel_9=0.0, pixel_10=0.0, pixel_11=0.0, pixel_12=0.0, pixel_13=0.0, pixel_14=0.0, pixel_15=0.0, pixel_16=0.0, pixel_17=0.0, pixel_18=0.0, pixel_19=0.0, pixel_20=0.0, pixel_21=0.0, pixel_22=0.0, pixel_23=0.0, pixel_24=0.0, pixel_25=0.0, pixel_26=0.0, pixel_27=0.0, pixel_28=0.0, pixel_29=0.0, pixel_30=0.0, pixel_31=0.0, pixel_32=0.0, pixel_33=0.0, pixel_34=0.0, pixel_35=0.0, pixel_36=0.0, pixel_37=0.0, pixel_38=0.0, pixel_39=0.0, pixel_40=0.0, pixel_41=0.0, pixel_42=0.0, pixel_43=0.0, pixel_44=0.0, pixel_45=0.0, pixel_46=0.0, pixel_47=0.0, pixel_48=0.0, pixel_49=0.0, pixel_50=0.0, pixel_51=0.0, pixel_52=0.0, pixel_53=0.0, pixel_54=0.0, pixel_55=0.0, pixel_56=0.0, pixel_57=

+-----+-----+
|label|count|
+-----+-----+
|   31| 3716|
|   53|11295|
|   34| 3837|
|   28|16573|
|   26| 2064|
|   27| 4044|
|   44| 2172|
|   12| 8041|
|   22| 7215|
|   47|12232|
|    1|30691|
|   52| 2383|
|   13| 3630|
|    6|27550|
|   16| 2003|
|    3|28151|
|   20| 1993|
|   40|19714|
|   57| 2325|
|   54| 2186|
+-----+-----+
only showing top 20 rows

None
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[Row(label=0, pixel_1=0.0, pixel_2=0.0, pixel_3=0.0, pixel_4=0.0, pixel_5=0.0, pixel_6=0.0, pixel_7=0.0, pixel_8=0.0, pixel_9=0.0, pixel_10=0.0, pixel_11=0.0, pixel_12=0.0, pixel_13=0.0, pixel_14=0.0, pixel_15=0.0, pixel_16=0.0, pixel_17=0.0, pixel_18=0.0, pixel_19=0.0, pixel_20=0.0, pixel_21=0.0, pixel_22=0.0, pixel_23=0.0, pixel_24=0.0, pixel_25=0.0, pixel_26=0.0, pixel_27=0.0, pixel_28=0.0, pixel_29=0.0, pixel_30=0.0, pixel_31=0.0, pixel_32=0.0, pixel_33=0.0, pixel_34=0.0, pixel_35=0.0, pixel_36=0.0, pixel_37=0.0, pixel_38=0.0, pixel_39=0.0, pixel_40=0.0, pixel_41=0.0, pixel_42=0.

+-----+-----+
|label|count|
+-----+-----+
|   31|  921|
|   53| 2810|
|   34|  906|
|   28| 4191|
|   26|  541|
|   27| 1029|
|   44|  553|
|   12| 2053|
|   22| 1787|
|   47| 3086|
|    1| 7683|
|   52|  611|
|   13|  932|
|    6| 6682|
|   16|  514|
|    3| 6992|
|   20|  475|
|   40| 4917|
|   57|  585|
|   54|  513|
+-----+-----+
only showing top 20 rows

None
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
[Row(label=36, pixel_1=0.0, pixel_2=0.0, pixel_3=0.0, pixel_4=0.0, pixel_5=0.0, pixel_6=0.0, pixel_7=0.0, pixel_8=0.0, pixel_9=0.0, pixel_10=0.0, pixel_11=0.0, pixel_12=0.0, pixel_13=0.0, pixel_14=0.0, pixel_15=0.0, pixel_16=0.0, pixel_17=0.0, pixel_18=0.0, pixel_19=0.0, pixel_20=0.0, pixel_21=0.0, pixel_22=0.0, pixel_23=0.0, pixel_24=0.0, pixel_25=0.0, pixel_26=0.0, pixel_27=0.0, pixel_28=0.0, pixel_29=0.0, pixel_30=0.0, pixel_31=0.0, pixel_32=0.0, pixel_33=0.0, pixel_34=0.0, pixel_35=0.0, pixel_36=0.0, pixel_37=0.0, pixel_38=0.0, pixel_39=0.0, pixel_40=0.0, pixel_41=0.0, pixel_42=0

+-----+-----+
|label|count|
+-----+-----+
|   31|  796|
|   53| 2320|
|   34|  798|
|   28| 3508|
|   26|  413|
|   27|  809|
|   44|  427|
|   12| 1739|
|   22| 1485|
|   47| 2535|
|    1| 6330|
|   52|  505|
|   13|  779|
|    6| 5705|
|   16|  447|
|    3| 5969|
|   40| 4092|
|   20|  382|
|   57|  468|
|   54|  437|
+-----+-----+
only showing top 20 rows

None


### Multi-layer Perceptron

In [14]:
# Defined the number of number of nodes in each layer (6 layers: 1 input, 4 hidden, 1 output)
layers1 = [784, 62, 62, 62, 62, 62]

# Assemble the row of pixel values into vector of pixel values using vector assembler
vector_assembler = VectorAssembler().setInputCols(train_newdf.columns[1:]).setOutputCol('features')

# Build the Perceptron classifier
mlp_classifier1 = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', predictionCol='prediction', maxIter=100, layers=layers1, blockSize=128, seed=0)
# Pipeline combining the vectorizer, and the perceptron
mlp_pipeline1 = Pipeline(stages=[vector_assembler, mlp_classifier1]).fit(train_newdf)

In [15]:
# Transform the validation set
mlp_validated_df1 = mlp_pipeline1.transform(validation_newdf)

# Print the prediction, actual label, and the features
print(mlp_validated_df1.select("prediction","label","features").show(5))

# Build evaluator to evaluate accuracy
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction', metricName="accuracy")
mlp_accuracy_validation1 = evaluator.evaluate(mlp_validated_df1)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|      37.0|    0|(784,[215,216,217...|
|       0.0|    0|(784,[212,213,214...|
|       0.0|    0|(784,[209,210,211...|
|       0.0|    0|(784,[208,209,210...|
|       0.0|    0|(784,[207,208,209...|
+----------+-----+--------------------+
only showing top 5 rows

None


In [37]:
# Achieved 63% accuracy on validation set
print("Accuracy on Validation 1:")
print(mlp_accuracy_validation1*100)

Accuracy on Validation 1:
63.00052318872778


In [18]:
# Save the pipeline
mlp_pipeline1.save("MLP_model")

In [19]:
# Second iteration with 12 layers (1 input, 10 hidden, 1 output)
layers2 = [784, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62]

# Pipeline for the second iteration
mlp_classifier2 = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', predictionCol='prediction', maxIter=100, layers=layers2, blockSize=128, seed=0)
mlp_pipeline2 = Pipeline(stages=[vector_assembler, mlp_classifier2]).fit(train_newdf)

In [20]:
# Transform the validation set
mlp_validated_df2 = mlp_pipeline2.transform(validation_newdf)

print(mlp_validated_df2.select("prediction","label","features").show(5))
mlp_accuracy_validation2 = evaluator.evaluate(mlp_validated_df2)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       1.0|    0|(784,[215,216,217...|
|       1.0|    0|(784,[212,213,214...|
|       1.0|    0|(784,[209,210,211...|
|       1.0|    0|(784,[208,209,210...|
|       1.0|    0|(784,[207,208,209...|
+----------+-----+--------------------+
only showing top 5 rows

None


In [36]:
# Achieved 5.5% accuracy
print("Accuracy on Validation 2:")
print(mlp_accuracy_validation2*100)

Accuracy on Validation 2:
5.506382185782168


The neural network performed worse with more hidden layers, most likely because it might be causing a lot of overfitting on the training set (Instead of neural network learning, it started memorizing the digits).

In [22]:
# Saved second pipeline
mlp_pipeline2.save("MLP_model2")

In [28]:
# Transform the test set with the first pipeline
test_final_mlp = mlp_pipeline1.transform(test_scaled)

# Print the prediction, actual label, and the features
print(test_final_mlp.select("prediction","label","features").show(5))

# Evaluate the accuracy on test set
test_accuracy_mlp = evaluator.evaluate(test_final_mlp)

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|      39.0|   36|(784,[72,73,74,75...|
|       0.0|    0|(784,[102,103,104...|
|       3.0|    3|(784,[88,89,90,11...|
|      33.0|   33|(784,[107,108,109...|
|      30.0|   30|(784,[99,100,101,...|
+----------+-----+--------------------+
only showing top 5 rows

None


In [34]:
# Achieved 62.9% accuracy on test set
print("Accuracy on testing set:")
print(test_accuracy_mlp*100)

Accuracy on testing set:
62.92790701672942


### Evaluation

In [55]:
# Select the prediction labels
pred_labels = test_final_mlp.select([c for c in test_final_mlp.columns if c in ["label","prediction"]]).toPandas()

In [57]:
# Build confusion matrix of actual labels and prediction
confusion_matrix_mlp = pd.crosstab(pred_labels["label"],pred_labels["prediction"], rownames=["Actual"],colnames=["Predicted"],margins=True)

In [70]:
confusion_matrix_mlp

Predicted,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,49.0,51.0,52.0,53.0,55.0,57.0,58.0,59.0,61.0,All
Actual,,,,,,,,,,,,,,,,,,,,,
0,4558,1,29,1,8,25,25,5,10,2,...,55,0,0,1,3,0,0,0,0,5778
1,0,5989,11,11,7,2,2,10,40,3,...,1,0,0,5,50,0,0,1,2,6330
2,17,10,5162,47,8,2,10,14,55,2,...,12,0,0,2,6,0,1,0,8,5869
3,1,16,101,5307,2,100,0,67,77,54,...,6,0,1,0,2,0,0,0,0,5969
4,0,1,7,0,4795,5,31,0,5,119,...,1,0,2,14,197,0,0,0,0,5619
5,21,13,4,106,12,3722,18,5,174,30,...,17,0,0,111,8,0,0,0,1,5190
6,1,0,13,0,13,39,4994,0,15,0,...,1,0,0,0,3,0,0,0,0,5705
7,1,13,17,19,24,1,0,5535,8,288,...,2,1,0,6,12,0,0,0,0,6139
8,2,40,38,73,7,106,8,5,4865,32,...,5,0,0,46,47,0,0,0,0,5633


### Live testing

In [4]:
In this testing function, we are reading the image (handwritten by the user) and bring it to the format readable by our model
The data is mad einto a dataframe as a column initially so transposed it to make it into a row wise dataframe:

img=Image.open("test_1.png").convert("1")
np_img=np.array(img.getdata())

img_pd=pd.DataFrame(np_img).T
img_pd.columns=col_list[1:]

In [5]:
#Scaling the pixel values from 0-255 to 0-1:

img_sp=spark.createDataFrame(img_pd)
scaled_test_df=img_sp
spark_test_list=col_list[1:]
for c in spark_test_list:
    name=str(c)
    scaled_test_df=scaled_test_df.withColumn(name,fn.col(name)/255)

In [6]:
#Finally using the best random forest model to get a prediction:
test_df=mlp_pipeline1.transform(scaled_test_df)

In [7]:
pred_df=test_df.select(fn.col("prediction")).toPandas()

In [10]:
Based on the label prediction, printing the actual character from char_list:

prediction=pred_df.iloc[0,0]
print(char_list[int(prediction)])

3
